In [54]:
# Importiamo le librerie

from itertools import combinations
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from itertools import combinations
from os import listdir
from os.path import isfile, join

In [55]:
# Variabili globali

mypath = 'dataset'

# Mi vado a prendere i path di tutti i file nella cartella dataset

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

# Mi vado a generare tutte le possibili combinazioni di due elementi da un insieme di 5 elementi


comb = list(combinations(range(0,5), 2))

In [56]:
# Andiamo a selezionare il file d'interesse

document = 'Friday-02-03-2018_TrafficForML_CICFlowMeter'
idx = 0

for i, j in enumerate(onlyfiles):
    if j == document:
        idx = i

folder = onlyfiles[idx]
folder = folder.replace('.csv','')
print(folder)

Friday-02-03-2018_TrafficForML_CICFlowMeter


In [57]:
# Andiamo a pulire tutti i dataset

# Come prima cosa andiamo a leggere le feature dai vari file csv

anova = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/anova.csv")
tree = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/ExtraTree.csv")
svm = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/SVM.csv")
rfe = pd.read_csv('top_feature/'+'Friday-23-02-2018_TrafficForML_CICFlowMeter'+"/RFE.csv")


In [58]:
# Andiamo a normalizzare tutti gli score delle feature, in maniera tale da avere valori compresi tra 0 e 1

anova['F_Score'] = anova['F_Score']/anova['F_Score'].max()
tree['F_Score'] = tree['F_Score']/tree['F_Score'].max()
svm['F_Score'] = svm['F_Score']/svm['F_Score'].max()
rfe['F_Score'] = rfe['F_Score']/rfe['F_Score'].max()

In [59]:
display(anova)

,Input_Features,F_Score
0,TotLen Fwd Pkts,1.000000
1,Subflow Fwd Byts,1.000000
2,Fwd Act Data Pkts,0.716001
3,Pkt Len Min,0.413904
4,Fwd Pkt Len Min,0.381537
5,Fwd Pkt Len Mean,0.193404
6,Fwd Seg Size Avg,0.193404
7,Idle Std,0.123829
8,Bwd Pkt Len Mean,0.081118
9,Bwd Seg Size Avg,0.081118


In [60]:
# Facciamo una concatenazione dei vari dataframe.
# Saranno utilizzati per prendere le feature che si ripetono di più e che hanno i punteggi maggiori

total_feature = pd.concat([anova,tree,svm,rfe],ignore_index=True)
total_feature_count = pd.concat([anova,tree,svm,rfe],ignore_index=True)

In [61]:
# Andiamo a eseguire un operazione di group by, con successivo count delle occorrenze dei vari campi

total_feature_count = total_feature_count.groupby(['Input_Features']).count().reset_index().sort_values(by='F_Score',ascending=False)

# Andiamo a rinominare la colonna F_Score, che adesso contiene i valori di count

total_feature_count.rename(columns = {'F_Score':'Conto'}, inplace = True)
print(total_feature_count)

       Input_Features  Conto
5        ECE Flag Cnt      3
21  Init Fwd Win Byts      3
16    Fwd Pkt Len Std      2
2    Bwd Pkt Len Mean      2
25   Subflow Fwd Byts      2
24       RST Flag Cnt      2
23        Pkt Len Min      2
7        Flow IAT Max      2
19           Idle Std      2
1     Bwd Pkt Len Max      2
17         Fwd Pkts/s      2
0          Active Min      1
22        Pkt Len Max      1
20  Init Bwd Win Byts      1
18   Fwd Seg Size Avg      1
13    Fwd Pkt Len Max      1
15    Fwd Pkt Len Min      1
14   Fwd Pkt Len Mean      1
12        Fwd IAT Tot      1
11  Fwd Act Data Pkts      1
10        Flow Pkts/s      1
9        Flow IAT Std      1
8        Flow IAT Min      1
6       Flow Duration      1
4    Bwd Seg Size Avg      1
3     Bwd Pkt Len Std      1
26    TotLen Fwd Pkts      1


In [62]:
# Delle delle feature più frequenti mi interessano le prime dieci

most_frequent = total_feature_count[:10]

print(most_frequent)

       Input_Features  Conto
5        ECE Flag Cnt      3
21  Init Fwd Win Byts      3
16    Fwd Pkt Len Std      2
2    Bwd Pkt Len Mean      2
25   Subflow Fwd Byts      2
24       RST Flag Cnt      2
23        Pkt Len Min      2
7        Flow IAT Max      2
19           Idle Std      2
1     Bwd Pkt Len Max      2


In [63]:
# Ci serve ancora il parametro F_Scores, quindi creiamo un nuovo dataframe con queste righe.
# Abbiamo le dieci righe, però, sono ancora presenti le ridondanze che prima avevamo eliminato.

test = pd.DataFrame(data=[],columns=total_feature.columns)

for a in most_frequent['Input_Features']:
    test = pd.concat([test,total_feature[total_feature['Input_Features'] == a]])

print(test)

       Input_Features   F_Score
12       ECE Flag Cnt  0.174431
26       ECE Flag Cnt  0.510887
33       ECE Flag Cnt  1.000000
10  Init Fwd Win Byts  1.000000
29  Init Fwd Win Byts  0.408599
36  Init Fwd Win Byts  1.000000
20    Fwd Pkt Len Std  1.000000
30    Fwd Pkt Len Std  1.000000
8    Bwd Pkt Len Mean  0.081118
13   Bwd Pkt Len Mean  0.132329
1    Subflow Fwd Byts  1.000000
38   Subflow Fwd Byts  1.000000
11       RST Flag Cnt  0.180407
25       RST Flag Cnt  0.510887
3         Pkt Len Min  0.413904
27        Pkt Len Min  0.470133
14       Flow IAT Max  0.127150
34       Flow IAT Max  1.000000
7            Idle Std  0.123829
28           Idle Std  0.438420
21    Bwd Pkt Len Max  0.803214
32    Bwd Pkt Len Max  1.000000


In [64]:
# Creo una variabile all'interno del quale vado a salvare il groupby del precedente dataframe, solo che in uesto caso
# andiamo a sommare i punteggi di F_Scores, invece di contare le occorrenze.

group_by = test.groupby(['Input_Features'])['F_Score'].sum().reset_index()

In [65]:
print(group_by)

      Input_Features   F_Score
0    Bwd Pkt Len Max  1.803214
1   Bwd Pkt Len Mean  0.213447
2       ECE Flag Cnt  1.685318
3       Flow IAT Max  1.127150
4    Fwd Pkt Len Std  2.000000
5           Idle Std  0.562249
6  Init Fwd Win Byts  2.408599
7        Pkt Len Min  0.884037
8       RST Flag Cnt  0.691294
9   Subflow Fwd Byts  2.000000


In [66]:
# Facciamo un ordinamento del dataframe precedente

print(group_by.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
6  Init Fwd Win Byts  2.408599
4    Fwd Pkt Len Std  2.000000
9   Subflow Fwd Byts  2.000000
0    Bwd Pkt Len Max  1.803214
2       ECE Flag Cnt  1.685318
3       Flow IAT Max  1.127150
7        Pkt Len Min  0.884037
8       RST Flag Cnt  0.691294
5           Idle Std  0.562249
1   Bwd Pkt Len Mean  0.213447


In [67]:
# A questo punto, terminiamo l'operazione andando a moltiplicare la frequenza per la somma dei punteggi delle varie feature.

last_feature = pd.DataFrame(data=[],columns=group_by.columns)

for a,b in zip(most_frequent['Input_Features'],most_frequent['Conto']):
    last_feature.loc[len(last_feature)] =  {'Input_Features':a,'F_Score':group_by[group_by['Input_Features'] == a]['F_Score'].values[0]*b}
    

In [68]:
# Facciamo una stampa ordinata

print(last_feature.sort_values(by='F_Score',ascending=False))

      Input_Features   F_Score
1  Init Fwd Win Byts  7.225796
0       ECE Flag Cnt  5.055954
2    Fwd Pkt Len Std  4.000000
4   Subflow Fwd Byts  4.000000
9    Bwd Pkt Len Max  3.606428
7       Flow IAT Max  2.254299
6        Pkt Len Min  1.768074
5       RST Flag Cnt  1.382588
8           Idle Std  1.124498
3   Bwd Pkt Len Mean  0.426894


In [69]:
# A questo punto ci interessano solo le prime 5 feature, e tutte le loro possibili combinazioni di due elementi.

esito_finale = pd.DataFrame(data=[],columns=['Feature_1','Feature_2'])
top_index = []

# Qui andiamo a prendere gli indici delle prime 5 features

for a in range(0,5):
    top_index.append(last_feature.sort_values(by='F_Score',ascending=False).index[a])
    
# Andiamo a inserire tutte le possibili combinazioni nel dataframe

for b in comb:

    esito_finale.loc[len(esito_finale)] =  {
        'Feature_1': last_feature.loc[top_index[b[0]]]['Input_Features'],
        'Feature_2': last_feature.loc[top_index[b[1]]]['Input_Features']
        }

In [70]:
print(esito_finale)

           Feature_1         Feature_2
0  Init Fwd Win Byts      ECE Flag Cnt
1  Init Fwd Win Byts   Fwd Pkt Len Std
2  Init Fwd Win Byts  Subflow Fwd Byts
3  Init Fwd Win Byts   Bwd Pkt Len Max
4       ECE Flag Cnt   Fwd Pkt Len Std
5       ECE Flag Cnt  Subflow Fwd Byts
6       ECE Flag Cnt   Bwd Pkt Len Max
7    Fwd Pkt Len Std  Subflow Fwd Byts
8    Fwd Pkt Len Std   Bwd Pkt Len Max
9   Subflow Fwd Byts   Bwd Pkt Len Max


In [71]:
# Salviamo l'esito finale in un file csv

esito_finale.to_csv('top_feature/'+folder+'/combinazioni.csv',index=False)